In [2]:
from IPython.display import HTML
from brax import envs
from brax.io import html
import jax
from frankenstein.algorithms.jax_brax_sac.utils import load_params
from frankenstein.algorithms.jax_brax_sac.networks import make_inference_fn, make_sac_networks
from frankenstein.algorithms.jax_brax_sac.running_statistics import normalize

In [10]:
env_name = "halfcheetah"
exp_name = "SAC"
timestep = "1000320"
backend = "generalized"

env = envs.get_environment(env_name=env_name, backend=backend)
path_to_save_model = f"/home/ttournai/Workspace/DAR/Frankenstein/runs/{env_name}/{exp_name}/model_{timestep}.pkl"
params = load_params(path_to_save_model)

FileNotFoundError: [Errno 2] No such file or directory: '/runs/halfcheetah/SAC/model_1000320.pkl'

In [4]:
actor_layers = (256, 256)
critic_layers = (256, 256)
normalize_fn = normalize  # normalize obs

sac_network = make_sac_networks(
    observation_size=env.observation_size,
    action_size=env.action_size,
    preprocess_observations_fn=normalize_fn,
    actor_layers=actor_layers,
    critic_layers=critic_layers,
)

2023-10-24 12:29:08.604256: W external/xla/xla/service/gpu/buffer_comparator.cc:1054] INTERNAL: ptxas exited with non-zero error code 65280, output: ptxas /tmp/tempfile-ttournai-ThinkStation-P350-fafbf18b-122829-60873cf11cb97, line 10; fatal   : Unsupported .version 7.8; current version is '7.3'
ptxas fatal   : Ptx assembly aborted due to errors

Relying on driver to perform ptx compilation. 
Setting XLA_FLAGS=--xla_gpu_cuda_data_dir=/path/to/cuda  or modifying $PATH can be used to set the location of ptxas
This message will only be logged once.


In [5]:
# Policy inference
make_policy = make_inference_fn(sac_network)
inference_fn = make_policy(params, deterministic=True)

In [6]:
env = envs.create(env_name=env_name, backend=backend)

jit_env_reset = jax.jit(env.reset)
jit_env_step = jax.jit(env.step)
jit_inference_fn = jax.jit(inference_fn)

rollouts, returns = [], []
episode_length = 1000
rng = jax.random.PRNGKey(seed=1)
state = jit_env_reset(rng=rng)
for _ in range(episode_length):
    rollouts.append(state.pipeline_state)
    returns.append(state.reward)
    act_rng, rng = jax.random.split(rng)
    act, _ = jit_inference_fn(state.obs, act_rng)
    state = jit_env_step(state, act)

In [7]:
#### Return
returns = [float(i) for i in returns]
print("Episode return == ", sum(returns))

Episode return ==  18720.63894033432


## Visualization

In [8]:
HTML(html.render(env.sys.replace(dt=env.dt), rollouts))